# Dataset Construction

In [1]:
import numpy as np
import pandas as pd
from torch_geometric_temporal.signal import StaticGraphTemporalSignalBatch, DynamicGraphTemporalSignalBatch
import torch
from typing import Union
import glob
from natsort import natsorted
import random

random.seed(42)

class SP500CorrelationsDatasetLoader(object):
    def __init__(self, corr_name, corr_scope):
        self._read_csv(corr_name, corr_scope)

    def _read_csv(self, corr_name, corr_scope):
        match corr_scope:
            case 'global':
                self._correlation_matrices = [np.loadtxt(f'{corr_name}/{corr_scope}_corr.csv', delimiter=',')]
            case 'local':
                self._correlation_matrices = []
                corr_files = natsorted(glob.glob(f'{corr_name}/local_corr_*.csv'))
                for corr_file in corr_files:
                    matrix = np.loadtxt(corr_file, delimiter=',')
                    self._correlation_matrices.append(matrix)
        
        # Thesholding
        _correlation_threshold = 0.9
        for matrix in self._correlation_matrices:
            matrix[matrix < _correlation_threshold] = 0
        
        df = pd.read_csv('s&p500.csv')
        df = df.set_index('Date')
        data = torch.from_numpy(df.to_numpy()).to(torch.float32)

        # Round data size to nearest multiple of batch_size
        self.days_in_quarter = 64
        num_quarters = data.size(0) // self.days_in_quarter
        num_days = num_quarters * self.days_in_quarter
        data = data[:num_days]
        
        # z-score normalization with training data following GERU
        train_days = int(0.8 * num_quarters) * self.days_in_quarter
        data = (data - data[:train_days].mean(dim=0)) / data[:train_days].std(dim=0)
        data = data.numpy()
        np.savetxt('s&p500_z_scores.csv', data, delimiter=',')
        self._dataset = data

    def _get_edges(self):
        if len(self._correlation_matrices) == 1:
            self._edges = np.array(self._correlation_matrices[0].nonzero())
        else:
            self._edges = []
            for time in range(self._dataset.shape[0] - self.lags):
                corr_index = time // self.days_in_quarter
                self._edges.append(
                    np.array(self._correlation_matrices[corr_index].nonzero())
                )

    def _get_edge_weights(self):
        if len(self._correlation_matrices) == 1:
            self._edge_weights = self._correlation_matrices[0][self._correlation_matrices[0] > 0]
        else:
            self._edge_weights = []
            for time in range(self._dataset.shape[0] - self.lags):
                corr_index = time // self.days_in_quarter
                self._edge_weights.append(
                    np.array(self._correlation_matrices[corr_index][self._correlation_matrices[corr_index] > 0])
                )

    def _get_targets_and_features(self):
        stacked_target = self._dataset
        self.features = [
            stacked_target[i : i + self.lags, :].T
            for i in range(stacked_target.shape[0] - self.lags)
        ]
        # predict next-day stock movement
        self.targets = [
            ((stacked_target[i + self.lags, :] > stacked_target[i + self.lags - 1, :]).astype(float)).T
            for i in range(stacked_target.shape[0] - self.lags)
        ]

    def get_dataset(self) -> Union[StaticGraphTemporalSignalBatch, DynamicGraphTemporalSignalBatch]:
        """Returning the data iterator.
        """
        self.lags = self.days_in_quarter
        self._get_edges()
        self._get_edge_weights()
        self._get_targets_and_features()
        self.batches = np.repeat(np.arange((self._dataset.shape[0] - self.lags) // self.days_in_quarter), self.days_in_quarter)
        dataset = (DynamicGraphTemporalSignalBatch if type(self._edges) == list else StaticGraphTemporalSignalBatch)(
            self._edges, self._edge_weights, self.features, self.targets, self.batches
        )
        return dataset

In [2]:
from torch_geometric_temporal.signal import temporal_signal_split

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'

corr_name = 'mi'
corr_scope = 'global'
loader = SP500CorrelationsDatasetLoader(corr_name=corr_name, corr_scope=corr_scope)

dataset = loader.get_dataset()
lags = loader.lags

train_dataset, test_val_dataset = temporal_signal_split(dataset, train_ratio=0.8)
val_dataset, test_dataset = temporal_signal_split(test_val_dataset, train_ratio=0.5)

# Evaluation

In [3]:
from torcheval.metrics.functional import binary_f1_score, binary_accuracy

def accuracy(y_hats, ys):
    return binary_accuracy(y_hats.flatten(), ys.flatten()).item()

def f1(y_hats, ys):
    return binary_f1_score(y_hats.flatten(), ys.flatten()).item()


# RGCN

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 64, 1)
        self.linear = torch.nn.Linear(64, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        return F.sigmoid(self.linear(h))

In [5]:
from tqdm import tqdm
import wandb

model = RecurrentGCN(node_features = lags).to(device)

lr = 1e-3
num_epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
track_with_wandb = True

if track_with_wandb:
    wandb.init(project="cs224w-stock-market-prediction", config={
        "dataset": "S&P500",
        "corr_name": corr_name,
        "corr_scope": corr_scope,
        "learning_rate": lr,
        "epochs": num_epochs,
        "architecture": "DCRNN",
    })

best_f1 = 0

train_samples = list(train_dataset)

for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    random.shuffle(train_samples)
    for time, snapshot in enumerate(train_samples):
        y_hat = model(snapshot.x.to(device), snapshot.edge_index.to(device), snapshot.edge_attr.to(device))
        loss = F.binary_cross_entropy(y_hat.squeeze(), snapshot.y)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    train_loss /= (time+1)

    if track_with_wandb:
        model.eval()
        with torch.no_grad():
            val_y_hats, val_ys = zip(*[(model(snapshot.x.to(device), snapshot.edge_index.to(device), snapshot.edge_attr.to(device)), snapshot.y)
                       for time, snapshot in enumerate(val_dataset)])
            val_y_hats, val_ys = torch.stack(list(val_y_hats)).squeeze().to(device), torch.stack(list(val_ys)).to(device)
            val_acc = accuracy(val_y_hats, val_ys)
            val_f1 = f1(val_y_hats, val_ys)
            wandb.log({"epoch": epoch,
                    "train/loss": train_loss,
                    "val/acc": val_acc,
                    "val/f1": val_f1 })
            print(f'Epoch {epoch}, val/acc: {val_acc}, val/f1: {val_f1}')
            if val_f1 > best_f1:
                best_f1 = val_f1
                torch.save(model.state_dict(), 'dcrnn_best_model.pth')


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kevinxli. Use `wandb login --relogin` to force relogin


  5%|▌         | 1/20 [00:10<03:28, 10.95s/it]

Epoch 0, val/acc: 0.5120143890380859, val/f1: 0.6772575974464417


 10%|█         | 2/20 [00:22<03:18, 11.03s/it]

Epoch 1, val/acc: 0.4892672896385193, val/f1: 0.22270873188972473


...

Epoch 17, val/acc: 0.5130518674850464, val/f1: 0.6699523329734802


 95%|█████████▌| 19/20 [03:29<00:11, 11.23s/it]

Epoch 18, val/acc: 0.512607216835022, val/f1: 0.6728352308273315


100%|██████████| 20/20 [03:40<00:00, 11.03s/it]

Epoch 19, val/acc: 0.5126595497131348, val/f1: 0.6717752814292908


In [6]:
if track_with_wandb:
    best_model = RecurrentGCN(node_features = lags).to(device)
    best_model.load_state_dict(torch.load('dcrnn_best_model.pth', weights_only=True))
    best_model.eval()
    with torch.no_grad():
        y_hats, ys = zip(*[(best_model(snapshot.x.to(device), snapshot.edge_index.to(device), snapshot.edge_attr.to(device)), snapshot.y)
                       for time, snapshot in enumerate(test_dataset)])
        y_hats, ys = torch.stack(list(y_hats)).squeeze().to(device), torch.stack(list(ys)).to(device)
        wandb.log({"epoch": epoch,
                "test/acc": accuracy(y_hats, ys),
                "test/f1": f1(y_hats, ys) })